# Clustering and Visulization
## using sentence_transformers
---
This is a simple application for sentence embeddings: clustering
Sentences are mapped to sentence embeddings and then multiple clustering algorithms are applied

In [9]:
from typing import Dict

mag_field_dict: Dict = {
    "Medicine"             : 0,
    "Biology"              : 1,
    "Chemistry"            : 2,
    "null"                 : 3,  # n/a
    "Engineering"          : 4,
    "Computer Science"     : 5,
    "Physics"              : 6,
    "Materials Science"    : 7,
    "Mathematics"          : 8,
    "Psychology"           : 9,
    "Economics"            : 10,
    "Political Science"    : 11,
    "Business"             : 12,
    "Geology"              : 13,
    "Sociology"            : 14,
    "Geography"            : 15,
    "Environmental Science": 16,
    "Art"                  : 17,
    "History"              : 18,
    "Philosophy"           : 19
}

In [10]:
from sentence_transformers import SentenceTransformer

In [1]:
%store -r s2orc_chunk

We count per field how many elements we have in the dataset

In [ ]:
counter_mags = dict()

In [29]:
def fill_train_examples_per_class(example, per_class_limit: int, counter: collections.Counter):
    label = int(example['label'])
    current_counter = counter.get(label, 0)
    if current_counter < per_class_limit:
        counter[label] = current_counter + 1
        return True
    return False

NameError: name 'collections' is not defined

['abstract', 'mag_field_of_study', 'title']

{'abstract': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'mag_field_of_study': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

AttributeError: 'Sequence' object has no attribute 'names'

/home/vivoli/miniconda3/envs/thesis/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
test_

In [ ]:
corpus = s2orc_chunk['title']

In [ ]:
embedder = SentenceTransformer('paraphrase-distilroberta-base-v1')

%time corpus_embeddings = embedder.encode(corpus, show_progress_bar=True)

# Visualization
---
Here we cover the code for visualizing the clusters generated by kmeans, hierarchical and fast-clustering

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import pandas as pd

In [ ]:
def plot_2d(red_algorithm, num_clusters, x, y, label):
    # print
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1)
    ax.set_xlabel(f'{red_algorithm} 1', fontsize = 15)
    ax.set_ylabel(f'{red_algorithm} 2', fontsize = 15)
    ax.set_title(f'2 Component {red_algorithm}', fontsize = 20)

    from matplotlib.lines import Line2D
    
    targets = [f'Cluster {i}' for i in range(num_clusters)] # + ['Centroids']
    markers = list(Line2D.markers.keys())[:num_clusters] # + ['$c$']
    
    print(f"targets: {targets}")
    print(f"markers: {markers}")
    

    for idx, (target, marker) in enumerate(zip(targets, markers)):
        indicesToKeep = label == idx # finalDf['target'] == idx
        ax.scatter(x[indicesToKeep], # finalDf.loc[indicesToKeep, f'{red_algorithm} 1'],
                   y[indicesToKeep], # finalDf.loc[indicesToKeep, f'{red_algorithm} 2'],
                   marker = marker,
                   # c = color,
                   s = 50)
    ax.legend(targets)
    ax.grid()
    plt.savefig(os.path.join(OUT_PATH, 'imgs', 'umap_hdbscan_umap.png'))

In [ ]:
def plot_3d(red_algorithm, num_clusters, x, y, z, label):
    # print
    fig = plt.figure(figsize = (8,8))
    ax = plt.axes(projection='3d')
    # ax = fig.add_subplot(1,1,1)
    ax.set_xlabel(f'{red_algorithm} 1', fontsize = 15)
    ax.set_ylabel(f'{red_algorithm} 2', fontsize = 15)
    ax.set_zlabel(f'{red_algorithm} 3', fontsize = 15)
    ax.set_title(f'3 Component {red_algorithm}', fontsize = 20)

    from matplotlib.lines import Line2D
    
    targets = [f'Cluster {i}' for i in range(num_clusters)] #  + ['Centroids']
    markers = list(Line2D.markers.keys())[:num_clusters] # + ['$c$']
    
    print(f"targets: {targets}")
    print(f"markers: {markers}")

    for idx, (target, marker) in enumerate(zip(targets, markers)):
        indicesToKeep = label == idx # finalDf['target'] == idx
        ax.scatter(x[indicesToKeep], # finalDf.loc[indicesToKeep, f'{red_algorithm} 1'],
                   y[indicesToKeep], # finalDf.loc[indicesToKeep, f'{red_algorithm} 2'],
                   z[indicesToKeep], # finalDf.loc[indicesToKeep, f'{red_algorithm} 3'],
                   marker = marker,
                   # c = color,
                   s = 50)
    ax.legend(targets)
    ax.grid()
    plt.savefig(os.path.join(OUT_PATH, 'imgs', 'umap_hdbscan_umap.png'))

In [ ]:
# scaler
from sklearn.preprocessing import StandardScaler

# dimensionality reduction
from sklearn.decomposition import PCA
from umap import UMAP
try:
    from MulticoreTSNE import MulticoreTSNE as TSNE
except:
    print("For reasonable computation time, install Multicore-TSNE!")
    from sklearn.manifold import TSNE

In [ ]:
def plot(red_algorithm, plot_dim, num_clusters, vectors, _labels):

    # print(f"***     Plot {plot_dim}D      ***")
    # perfect_half_spaces = ''*(10-len(red_algorithm))
    # print(f"* {perfect_half_spaces + red_algorithm + perfect_half_spaces} *")
    # print(f"num_clusters: {num_clusters}")
    # print(f"vectors (shape): {vectors.shape}")
    # print(f"_labels (shape): {_labels.shape}")
    # print("*"*25)
        
    _labels = _labels
    _vectors = StandardScaler().fit_transform(vectors)
    
    if red_algorithm == 'pca':
        plot_dim = 3
        algorithm = PCA(n_components=plot_dim)
    elif red_algorithm == 'tsne':
        algorithm = TSNE(n_components=plot_dim)
    elif red_algorithm == 'umap':
        algorithm = UMAP(n_components=plot_dim)
    else:
        raise ValueError(f"The PLOT algorithm {red_algorithm} is not supported, yet!")
    
    red_vectors = algorithm.fit_transform(_vectors)

    # dataframes from pandas
    principalDf = pd.DataFrame(data = red_vectors, columns = [f'{red_algorithm} {num+1}' for num in range(plot_dim)])
    labelDf =  pd.DataFrame(data = _labels, columns = ['target'])
    finalDf = pd.concat([principalDf, labelDf], axis = 1)
    
    if red_algorithm == 'pca':
        plot_2d(red_algorithm, num_clusters, finalDf[f'{red_algorithm} 1'], finalDf[f'{red_algorithm} 2'], finalDf['target'])
        plot_3d(red_algorithm, num_clusters, finalDf[f'{red_algorithm} 1'], finalDf[f'{red_algorithm} 2'], finalDf[f'{red_algorithm} 3'], finalDf['target'])
    else:
        if plot_dim == 2:
            plot_2d(red_algorithm, num_clusters, finalDf[f'{red_algorithm} 1'], finalDf[f'{red_algorithm} 2'], finalDf['target'])
        elif plot_dim == 3:
            plot_3d(red_algorithm, num_clusters, finalDf[f'{red_algorithm} 1'], finalDf[f'{red_algorithm} 2'], finalDf[f'{red_algorithm} 3'], finalDf['target'])
        else:
            raise ValueError(f"plot_dim not supported {plot_dim} !!")

# K-means
---
Use of k-mean clustering.
Sentences are mapped to sentence embeddings and then k-mean clustering is applied.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Perform kmean clustering
num_clusters = 20
clustering_model = KMeans(n_clusters=num_clusters)
%time clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [ ]:
s2orc_chunk['mag_field_of_study'][0]

In [ ]:
corpus_embeddings

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

In [ ]:
clustered_mags = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_mags[cluster_id] += s2orc_chunk['mag_field_of_study'][sentence_id]   

In [ ]:
clustered_mags

In [ ]:
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i)
    print(cluster[:10])
    print("")

In [ ]:
vectors = corpus_embeddings
_labels = clustering_model.labels_

In [ ]:
algorithms = ['pca', 'tsne', 'umap'] 
dimentions = [ 2, 3 ]

for red_algorithm in algorithms:
    plot_algorithm(red_algorithm, plot_dim, num_clusters, vectors, _labels)

# Hierarchical clustering
---
Use of Hierarchical clustering.
Sentences are mapped to sentence embeddings and then agglomerative clustering with a threshold is applied.
This algorithm can be useful `if the number of clusters is unknown`. By the threshold, we can control if we want to have many small and fine-grained cluster or few coarse-grained clusters.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

In [ ]:
# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [ ]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []
    clustered_sentences[cluster_id].append(corpus[sentence_id])

In [ ]:
for i, cluster in clustered_sentences.items():
    print("Cluster ", i)
    print(cluster)
    print("")

In [ ]:
num_clusters = len(clustered_sentences)
vectors = corpus_embeddings
_labels = clustering_model.labels_

In [ ]:
algorithms = ['pca', 'tsne', 'umap'] 
dimentions = [ 2, 3 ]

for red_algorithm in algorithms:
    for plot_dim in dimentions:
        plot(red_algorithm, plot_dim, num_clusters, vectors, _labels)

# Fast-Clustering
---
This is a more complex example on performing `clustering on large scale dataset`.
This examples find in a large set of sentences local communities, i.e., `groups of sentences that are highly similar`. 
You can freely configure the threshold what is considered as similar. 
A `high threshold` will only find `extremely similar sentences`, a `lower threshold` will find more sentence that are `less similar`.
A second parameter is `'min_community_size'`: Only communities with at least a certain number of sentences will be returned.
The method for finding the communities is extremely fast, for clustering `50k sentences it requires only 5 seconds` (plus embedding comuptation).
In this example, we download a large set of questions from Quora and then find similar questions in this set.

In [ ]:
from sentence_transformers import util
import numpy as np
import os
import csv
import pickle
import time

In [ ]:
def community_detection(embeddings, threshold=0.75, min_community_size=10, init_max_size=1000):
    """
    Function for Fast Community Detection
    Finds in the embeddings all communities, i.e. embeddings that are close (closer than threshold).
    Returns only communities that are larger than min_community_size. The communities are returned
    in decreasing order. The first element in each list is the central point in the community.
    """

    # Compute cosine similarity scores
    cos_scores = util.pytorch_cos_sim(embeddings, embeddings)

    # Minimum size for a community
    top_k_values, _ = cos_scores.topk(k=min_community_size, largest=True)

    # Filter for rows >= min_threshold
    extracted_communities = []
    for i in range(len(top_k_values)):
        if top_k_values[i][-1] >= threshold:
            new_cluster = []

            # Only check top k most similar entries
            top_val_large, top_idx_large = cos_scores[i].topk(k=init_max_size, largest=True)
            top_idx_large = top_idx_large.tolist()
            top_val_large = top_val_large.tolist()

            if top_val_large[-1] < threshold:
                for idx, val in zip(top_idx_large, top_val_large):
                    if val < threshold:
                        break

                    new_cluster.append(idx)
            else:
                # Iterate over all entries (slow)
                for idx, val in enumerate(cos_scores[i].tolist()):
                    if val >= threshold:
                        new_cluster.append(idx)

            extracted_communities.append(new_cluster)

    # Largest cluster first
    extracted_communities = sorted(extracted_communities, key=lambda x: len(x), reverse=True)

    # Step 2) Remove overlapping communities
    unique_communities = []
    extracted_ids = set()

    for community in extracted_communities:
        add_cluster = True
        for idx in community:
            if idx in extracted_ids:
                add_cluster = False
                break

        if add_cluster:
            unique_communities.append(community)
            for idx in community:
                extracted_ids.add(idx)

    return unique_communities

In [ ]:
# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer('distilbert-base-nli-stsb-quora-ranking')

In [ ]:
# We donwload the Quora Duplicate Questions Dataset (https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs)
# and find similar question in it
max_corpus_size = 50000 # We limit our corpus to only the first 50k questions
embedding_cache_path = '/home/vivoli/Thesis/data/clustering-samples/s2orc_corpus_sample-{}.pkl'.format(max_corpus_size)

In [ ]:
#Check if embedding cache path exists
if not os.path.exists(embedding_cache_path):
    # Check if the dataset exists. If not, download and extract
    # Download dataset if needed

    print("Encode the corpus. This might take a while")
    corpus_embeddings = model.encode(corpus, show_progress_bar=True, convert_to_numpy=True)

    print("Store file on disc")
    with open(embedding_cache_path, "wb") as fOut:
        pickle.dump({'sentences': corpus, 'embeddings': corpus_embeddings}, fOut)
else:
    print("Load pre-computed embeddings from disc")
    with open(embedding_cache_path, "rb") as fIn:
        cache_data = pickle.load(fIn)
        corpus_sentences = cache_data['sentences']
        corpus_embeddings = cache_data['embeddings']

In [ ]:
corpus_embeddings.shape

In [ ]:
print("Start clustering")
start_time = time.time()

# Two parameter to tune:
# min_cluster_size: Only consider cluster that have at least 25 elements (30 similar sentences)
# threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = community_detection(corpus_embeddings, min_community_size=25, threshold=0.2)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

In [ ]:
_len = 0
for cluster in clusters:
    _len += len(cluster)
print(_len)

In [ ]:
#Print all cluster / communities
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster:
        print("\t", corpus_sentences[sentence_id])

In [ ]:
num_clusters = len(clusters)
vectors = corpus_embeddings[np.asarray(np.concatenate(clusters).flat)]
_labels = np.asarray(np.concatenate([[idx]*len(cluster) for idx, cluster in enumerate(clusters)]).flat)

In [ ]:
algorithms = ['pca', 'tsne', 'umap'] 
dimentions = [ 2, 3 ]

for red_algorithm in algorithms:
    for plot_dim in dimentions:
        plot(red_algorithm, plot_dim, num_clusters, vectors, _labels)

# UMAP
---
---
Incredible: it works !

In [ ]:
# !conda install numpy scipy -y; conda install scikit-learn -y; conda install numba -y; pip install umap-learn
# !cd utils; wget https://github.com/lmcinnes/umap/archive/master.zip; unzip master.zip; rm master.zip; cd umap-master; conda install scikit-learn numba -y; python setup.py install
# !pip install scipy
# !cd utils; cd umap-master; pip install --user -r requirements.txt; python setup.py install --user
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import umap
%matplotlib inline

In [ ]:
np.random.seed(42)
data = np.random.rand(800, 4)

In [ ]:
fit = umap.UMAP()
%time u = fit.fit_transform(data)

In [ ]:
plt.scatter(u[:,0], u[:,1], c=data)
plt.title('UMAP embedding of random colours');

In [ ]:
def draw_umap(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean', title=''):
    fit = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric
    )
    u = fit.fit_transform(data);
    fig = plt.figure()
    if n_components == 1:
        ax = fig.add_subplot(111)
        ax.scatter(u[:,0], range(len(u)), c=data)
    if n_components == 2:
        ax = fig.add_subplot(111)
        ax.scatter(u[:,0], u[:,1], c=data)
    if n_components == 3:
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(u[:,0], u[:,1], u[:,2], c=data, s=100)
    plt.title(title, fontsize=18)

In [ ]:
for n in (2, 5, 10, 20, 50, 100, 200):
    draw_umap(n_neighbors=n, title='n_neighbors = {}'.format(n))

In [ ]:
for d in (0.0, 0.1, 0.25, 0.5, 0.8, 0.99):
    draw_umap(min_dist=d, title='min_dist = {}'.format(d))

In [ ]:
draw_umap(n_components=1, title='n_components = 1')

In [ ]:
draw_umap(n_components=3, title='n_components = 3')

In [30]:
import numba

In [31]:
@numba.njit()
def hue(r, g, b):
    cmax = max(r, g, b)
    cmin = min(r, g, b)
    delta = cmax - cmin
    if cmax == r:
        return ((g - b) / delta) % 6
    elif cmax == g:
        return ((b - r) / delta) + 2
    else:
        return ((r - g) / delta) + 4

@numba.njit()
def lightness(r, g, b):
    cmax = max(r, g, b)
    cmin = min(r, g, b)
    return (cmax + cmin) / 2.0

@numba.njit()
def saturation(r, g, b):
    cmax = max(r, g, b)
    cmin = min(r, g, b)
    chroma = cmax - cmin
    light = lightness(r, g, b)
    if light == 1:
        return 0
    else:
        return chroma / (1 - abs(2*light - 1))


In [32]:
@numba.njit()
def red_channel_dist(a,b):
    return np.abs(a[0] - b[0])

@numba.njit()
def sl_dist(a, b):
    a_sat = saturation(a[0], a[1], a[2])
    b_sat = saturation(b[0], b[1], b[2])
    a_light = lightness(a[0], a[1], a[2])
    b_light = lightness(b[0], b[1], b[2])
    return (a_sat - b_sat)**2 + (a_light - b_light)**2

@numba.njit()
def hue_dist(a, b):
    diff = (hue(a[0], a[1], a[2]) - hue(b[0], b[1], b[2])) % 6
    if diff < 0:
        return diff + 6
    else:
        return diff

@numba.njit()
def hsl_dist(a, b):
    a_sat = saturation(a[0], a[1], a[2])
    b_sat = saturation(b[0], b[1], b[2])
    a_light = lightness(a[0], a[1], a[2])
    b_light = lightness(b[0], b[1], b[2])
    a_hue = hue(a[0], a[1], a[2])
    b_hue = hue(b[0], b[1], b[2])
    return (a_sat - b_sat)**2 + (a_light - b_light)**2 + (((a_hue - b_hue) % 6) / 6.0)

In [33]:
for m in ("euclidean", red_channel_dist, sl_dist, hue_dist, hsl_dist):
    name = m if type(m) is str else m.__name__
    draw_umap(n_components=2, metric=m, title='metric = {}'.format(name))

NameError: name 'draw_umap' is not defined

# 3d interaction
---
---

In [ ]:
import ipyvolume as ipv
import numpy as np

In [ ]:
x, y, z = np.random.random((3, 10000))
ipv.quickscatter(x, y, z, size=1, marker="sphere")

In [ ]:
import numpy as np
import ipyvolume as ipv
V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0
ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)
